# Spark LLM Assistant

## Initialization

In [1]:
from langchain.chat_models import ChatOpenAI
from spark_llm import SparkLLMAssistant

llm = ChatOpenAI(model_name='gpt-4') # using gpt-4 can achieve better results
assistant=SparkLLMAssistant(llm=llm, verbose=True)
assistant.activate() # active partial functions for Spark DataFrame

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/15 22:16:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Example 1: Auto sales by brand in US 2022

In [5]:
# Search and ingest web content into a DataFrame
auto_df = assistant.create_df("2022 USA national auto sales by brand")
auto_df.show()

Parsing URL: https://www.carpro.com/blog/full-year-2022-national-auto-sales-by-brand

SQL query for the ingestion:
 CREATE OR REPLACE TEMP VIEW auto_sales_2022 AS SELECT * FROM VALUES
('Toyota', 1849751, -9),
('Ford', 1767439, -2),
('Chevrolet', 1502389, 6),
('Honda', 881201, -33),
('Hyundai', 724265, -2),
('Kia', 693549, -1),
('Jeep', 684612, -12),
('Nissan', 682731, -25),
('Subaru', 556581, -5),
('Ram Trucks', 545194, -16),
('GMC', 517649, 7),
('Mercedes-Benz', 350949, 7),
('BMW', 332388, -1),
('Volkswagen', 301069, -20),
('Mazda', 294908, -11),
('Lexus', 258704, -15),
('Dodge', 190793, -12),
('Audi', 186875, -5),
('Cadillac', 134726, 14),
('Chrysler', 112713, -2),
('Buick', 103519, -42),
('Acura', 102306, -35),
('Volvo', 102038, -16),
('Mitsubishi', 102037, -16),
('Lincoln', 83486, -4),
('Porsche', 70065, 0),
('Genesis', 56410, 14),
('INFINITI', 46619, -20),
('MINI', 29504, -1),
('Alfa Romeo', 12845, -30),
('Maserati', 6413, -10),
('Bentley', 3975, 0),
('Lamborghini', 3134, 3),
('Fi

In [11]:
auto_df.llm_plot()

You can visualize the result of the `df` DataFrame using Plotly in Python like this:

```python
import plotly.express as px
import plotly.io as pio

# Convert the Spark DataFrame to a Pandas DataFrame
pdf = df.toPandas()

# Create a bar chart using Plotly Express
fig = px.bar(pdf, x='brand', y='US_sales', text='sales_change', title='Auto Sales 2022')

# Show the plot directly
pio.show(fig)
```

In this code block, we first import the necessary Plotly libraries and then convert the Spark DataFrame `df` to a Pandas DataFrame, which is easier to work with for visualization purposes. We then use Plotly Express to create a bar chart with the 'brand' column on the x-axis, the 'US_sales' column on the y-axis, and the 'sales_change' column as text labels on the bars. Finally, we display the plot directly using the `pio.show()` function.


In [8]:
# Apply transforms to a Dataframe
auto_top_growth_df=auto_df.llm_transform("top brand with the highest growth")
auto_top_growth_df.show()

SQL query for the transform:
SELECT brand, sales_change
FROM temp_view_for_transform
ORDER BY sales_change DESC
LIMIT 1
+-------+------------+
|  brand|sales_change|
+-------+------------+
|Genesis|          14|
+-------+------------+



In [9]:
# Explain what a DataFrame is retrieving.
auto_top_growth_df.llm_explain()

'In summary, this dataframe is retrieving the brand with the highest sales change from the `temp_view_for_transform` view, which is based on the `auto_sales_2022` view. The result will show only one record with the brand and its corresponding sales change value.'

## Example 2: USA Presidents

In [12]:
# You can also specify the expected columns for the ingestion.
df=assistant.create_df("USA presidents", ["president", "vice_president"])
df.show()

Parsing URL: https://www.loc.gov/rr/print/list/057_chron.html

SQL query for the ingestion:
 CREATE OR REPLACE TEMP VIEW usa_presidents AS SELECT * FROM VALUES
('George Washington', 'John Adams'),
('John Adams', 'Thomas Jefferson'),
('Thomas Jefferson', 'Aaron Burr'),
('Thomas Jefferson', 'George Clinton'),
('James Madison', 'George Clinton'),
('James Madison', 'Elbridge Gerry'),
('James Monroe', 'Daniel D. Tompkins'),
('John Quincy Adams', 'John C. Calhoun'),
('Andrew Jackson', 'John C. Calhoun'),
('Andrew Jackson', 'Martin Van Buren'),
('Martin Van Buren', 'Richard M. Johnson'),
('William Henry Harrison', 'John Tyler'),
('John Tyler', NULL),
('James K. Polk', 'George M. Dallas'),
('Zachary Taylor', 'Millard Fillmore'),
('Millard Fillmore', NULL),
('Franklin Pierce', 'William R. King'),
('Franklin Pierce', NULL),
('James Buchanan', 'John C. Breckinridge'),
('Abraham Lincoln', 'Hannibal Hamlin'),
('Abraham Lincoln', 'Andrew Johnson'),
('Andrew Johnson', NULL),
('Ulysses S. Grant', 'Sch

In [13]:
presidents_who_were_vp = df.llm_transform("presidents who were also vice presidents")
presidents_who_were_vp.show()

SQL query for the transform:
SELECT DISTINCT president FROM temp_view_for_transform WHERE president IN (SELECT DISTINCT vice_president FROM temp_view_for_transform)
+------------------+
|         president|
+------------------+
|        John Adams|
|  Thomas Jefferson|
|  Martin Van Buren|
|  Millard Fillmore|
|        John Tyler|
|    Andrew Johnson|
| Chester A. Arthur|
|Theodore Roosevelt|
|   Calvin Coolidge|
|   Harry S. Truman|
|    Gerald R. Ford|
| Lyndon B. Johnson|
|  Richard M. Nixon|
|       George Bush|
|   Joseph R. Biden|
+------------------+



In [14]:
presidents_who_were_vp.llm_explain()

'In summary, this dataframe is retrieving the distinct list of presidents who have never served as a vice president.'

# Example 3: Top 10 tech companies

In [15]:
# Search and ingest web content into a DataFrame
company_df=assistant.create_df("Top 10 tech companies by market cap", ['company', 'cap', 'country'])
company_df.show()

Parsing URL: https://www.statista.com/statistics/1350976/leading-tech-companies-worldwide-by-market-cap/

SQL query for the ingestion:
 CREATE OR REPLACE TEMP VIEW top_tech_companies AS SELECT * FROM VALUES
('Apple', 2242, 'USA'),
('Microsoft', 1821, 'USA'),
('Alphabet (Google)', 1229, 'USA'),
('Amazon', 902.4, 'USA'),
('Tesla', 541.4, 'USA'),
('TSMC', 410.9, 'Taiwan'),
('NVIDIA', 401.7, 'USA'),
('Tencent', 377.8, 'China'),
('Meta Platforms (Facebook)', 302.1, 'USA'),
('Samsung', 301.7, 'South Korea')
AS v1(company, cap, country)

Storing data into temp view: top_tech_companies

+--------------------+------+-----------+
|             company|   cap|    country|
+--------------------+------+-----------+
|               Apple|2242.0|        USA|
|           Microsoft|1821.0|        USA|
|   Alphabet (Google)|1229.0|        USA|
|              Amazon| 902.4|        USA|
|               Tesla| 541.4|        USA|
|                TSMC| 410.9|     Taiwan|
|              NVIDIA| 401.7|       

In [16]:
us_company_df=company_df.llm_transform("companies in USA")
us_company_df.show()

SQL query for the transform:
SELECT company FROM temp_view_for_transform WHERE country = 'USA'
+--------------------+
|             company|
+--------------------+
|               Apple|
|           Microsoft|
|   Alphabet (Google)|
|              Amazon|
|               Tesla|
|              NVIDIA|
|Meta Platforms (F...|
+--------------------+



In [17]:
us_company_df.llm_explain()

'In summary, this dataframe is retrieving the company names of top technology companies located in the USA.'

In [18]:
us_company_df.llm_plot()

```python
import plotly.express as px
import pandas as pd

# Convert Spark DataFrame to Pandas DataFrame
pdf = df.toPandas()

# Create a bar chart using Plotly Express
fig = px.bar(pdf, x='company', y='company', color='company', text='company', title="USA Technology Companies", labels={'company': 'Company Name'})

# Show the plot
fig.show()
```
Note: This code assumes that you have already installed the Plotly library and have the necessary dependencies. If not, you can install it using `!pip install plotly` in a Jupyter Notebook or `pip install plotly` in a terminal. Also, note that Plotly might not render the chart directly in some environments (like plain Python scripts) and might require a Jupyter Notebook or another supported environment to display the chart.


## Example 4: Ingestion from a URL
Instead of searching for the web page, you can also ask the assistant to ingest from a URL.

In [21]:
assistant.create_df('https://time.com/6235186/best-albums-2022/').show()

Parsing URL: https://time.com/6235186/best-albums-2022/

SQL query for the ingestion:
 CREATE OR REPLACE TEMP VIEW best_albums_2022 AS SELECT * FROM VALUES
('Motomami, Rosalía', 10),
('You Can’t Kill Me, 070 Shake', 9),
('Mr. Morale & The Big Steppers, Kendrick Lamar', 8),
('Big Time, Angel Olsen', 7),
('Electricity, Ibibio Sound Machine', 6),
('It’s Almost Dry, Pusha T', 5),
('Chloe and the Next 20th Century, Father John Misty', 4),
('Renaissance, Beyoncé', 3),
('19 Masters, Saya Gray', 2),
('Un Verano Sin Ti, Bad Bunny', 1)
AS v1(album, rank);

Storing data into temp view: best_albums_2022

+--------------------+----+
|               album|rank|
+--------------------+----+
|   Motomami, Rosalía|  10|
|You Can’t Kill Me...|   9|
|Mr. Morale & The ...|   8|
|Big Time, Angel O...|   7|
|Electricity, Ibib...|   6|
|It’s Almost Dry, ...|   5|
|Chloe and the Nex...|   4|
|Renaissance, Beyoncé|   3|
|19 Masters, Saya ...|   2|
|Un Verano Sin Ti,...|   1|
+--------------------+----+



# Cache
The SparkLLMAssistant supports a simple in-memory and persistent cache system. It keeps an in-memory staging cache, which gets updated for LLM and web search results. The staging cache can be persisted through the commit() method. Cache lookup is always performed on the persistent cache only.

In [20]:
assistant.commit()